In [3]:
import pandas as pd

In [39]:
def vcf2dataframe(vcfpath, parse_info=False, parse_annot=False):
    '''
    读取一个vcf转换成dataframe 列与vcfheader的列一致
    '''
    if not parse_info and parse_annot:
        parse_info = True
        print('[WARNING] When parse_annot=True, parse_info will be forced set to True!')
    for line in open(vcfpath).readlines():
        if line.startswith('#CHROM'):
            vcfdfheader = line.lstrip('#').split()
            break
    vcfdf = pd.read_table(vcfpath, comment='#', header=None, names=vcfdfheader)
    if len(vcfdf)==0:
        return vcfdf
    if parse_info:
        vcfinfo = pd.DataFrame(list(vcfdf['INFO'].apply(lambda x: { pair.split('=')[0]:pair.split('=')[1] for pair in x.split(';') })))
        vcfdf = pd.concat([vcfdf, vcfinfo], axis=1)
        if parse_annot: # parse annotation from snpEff
            ann_cols = 'Allele | Annotation | Annotation_Impact | Gene_Name | Gene_ID | Feature_Type | Feature_ID | Transcript_BioType | Rank | HGVS.c | HGVS.p | cDNA.pos / cDNA.length | CDS.pos / CDS.length | AA.pos / AA.length | Distance | ERRORS / WARNINGS / INFO'.split(' | ')
            vcfann = pd.DataFrame(list(vcfdf['ANN'].apply(lambda x: x.split('|', 15))), columns = ann_cols)
            vcfdf = pd.concat([vcfdf.drop('ANN', axis=1), vcfann], axis=1)
    return vcfdf

In [28]:
isnv_full = pd.read_excel('/jdfssz1/ST_HEALTH/P20Z10200N0206/renzirui/EA_redo/ViralStrain_PopGenetics/iSNV_full_perPOS_annotated.xlsx')
isnv_full

,SN,CHROM,POS,REF,ALT,ADPP,Annotation,sn,vsname
0,R2011005870,PicoV-24A,4328,C,T,0.064286,missense_variant,R2011005870,PicoV-24A
1,R2011005870,PicoV-24A,5889,T,C,0.080645,synonymous_variant,R2011005870,PicoV-24A
2,R2011005891,PicoV-24A,660,A,C,0.101449,synonymous_variant,R2011005891,PicoV-24A
3,R2011005891,PicoV-24A,4587,T,C,0.304878,synonymous_variant,R2011005891,PicoV-24A
4,R2011005891,PicoV-24A,5448,T,C,0.122642,synonymous_variant,R2011005891,PicoV-24A
...,...,...,...,...,...,...,...,...,...
18916,R2101012916,CV-6A,1399,A,G,0.427350,missense_variant,R2101012916,CV-6A
18917,R2101012916,CV-6A,1405,A,T,0.356436,missense_variant,R2101012916,CV-6A
18918,R2101012916,CV-6A,1814,T,C,0.340659,synonymous_variant,R2101012916,CV-6A
18919,R2101012909,CV-6A,1387,C,A,0.160987,missense_variant,R2101012909,CV-6A


In [4]:
isnv_stat_filtdp = pd.read_excel('/jdfssz1/ST_HEALTH/P20Z10200N0206/renzirui/EA_redo/ViralStrain_PopGenetics/iSNV_DPstat_filtCoInf_filtDP_indv.xlsx')
isnv_stat_filtdp

,vsname,sn,num_iSNV,num_ALTiSNV,synonymous_ALTiSNV,synonymous_meanDP,nonsynonymous_ALTiSNV,nonsynonymous_meanDP,meandepth,coverage
0,CalV-11A,R2101013357,3,3,3,0.105905,0,NaN,322.8590,100.0000
1,CalV-11A,R2101013362,7,7,7,0.187506,0,NaN,628.6070,100.0000
2,CV-12A,R2011008079,5,5,1,0.076060,0,NaN,813.1420,100.0000
3,CV-12A,R2011008085,3,1,0,NaN,0,NaN,52.4100,74.5821
4,CV-12A,R2011008083,16,8,0,NaN,2,0.232190,443.6190,95.9685
...,...,...,...,...,...,...,...,...,...,...
95,ParV-1A,R2101013077,86,50,1,0.100070,0,NaN,1829.6200,86.7643
96,ReoV-2B,R2012012467,66,45,29,0.305499,1,0.076923,74.3356,98.1026
97,ReoV-2B,R2012012465,5,5,3,0.064791,2,0.070476,282.4040,100.0000
98,CV-6A,R2101012916,24,7,3,0.306084,4,0.316236,171.2540,100.0000


In [64]:
isnv_stat_filtdp[isnv_stat_filtdp['vsname']=='AdV-2A']

,vsname,sn,num_iSNV,num_ALTiSNV,synonymous_ALTiSNV,synonymous_meanDP,nonsynonymous_ALTiSNV,nonsynonymous_meanDP,meandepth,coverage
13,AdV-2A,R2011008078,15,13,2,0.089999,10,0.102204,305.9740,99.1281
14,AdV-2A,R2011008080,3,3,0,NaN,3,0.139277,86.4159,99.6952


In [25]:
isnv_stat_filtdp[isnv_stat_filtdp['vsname']=='ParV-1A']

,vsname,sn,num_iSNV,num_ALTiSNV,synonymous_ALTiSNV,synonymous_meanDP,nonsynonymous_ALTiSNV,nonsynonymous_meanDP,meandepth,coverage
80,ParV-1A,R2101012996,100,59,2,0.183448,0,NaN,589.786,89.0909
81,ParV-1A,R2101013052,102,82,0,NaN,0,NaN,7510.520,98.9368
82,ParV-1A,R2101013078,68,40,1,0.079365,0,NaN,3103.100,86.7488
83,ParV-1A,R2101013074,78,43,0,NaN,0,NaN,2110.260,86.8721
84,ParV-1A,R2101013049,88,50,0,NaN,0,NaN,2235.880,86.8721
85,ParV-1A,R2101013047,114,94,0,NaN,0,NaN,2475.130,100.0000
86,ParV-1A,R2101013044,80,44,0,NaN,0,NaN,409.437,86.7797
87,ParV-1A,R2101012993,75,60,0,NaN,0,NaN,352.639,90.6317
88,ParV-1A,R2101013072,66,39,1,0.059662,0,NaN,2340.500,86.8567
89,ParV-1A,R2101013045,91,52,0,NaN,0,NaN,1231.890,86.8413


In [10]:
isnv_kept = isnv_full.merge(isnv_stat_filtdp[['vsname', 'sn']], on=['vsname', 'sn'])
isnv_kept

,SN,CHROM,POS,REF,ALT,ADPP,Annotation,sn,vsname
0,R2101013357,CalV-11A,3405,C,T,0.145695,synonymous_variant,R2101013357,CalV-11A
1,R2101013357,CalV-11A,6156,C,T,0.101266,synonymous_variant,R2101013357,CalV-11A
2,R2101013357,CalV-11A,6189,C,T,0.070755,synonymous_variant,R2101013357,CalV-11A
3,R2101013362,CalV-11A,108,C,T,0.071563,synonymous_variant,R2101013362,CalV-11A
4,R2101013362,CalV-11A,1380,T,C,0.060311,synonymous_variant,R2101013362,CalV-11A
...,...,...,...,...,...,...,...,...,...
1658,R2101012916,CV-6A,1399,A,G,0.427350,missense_variant,R2101012916,CV-6A
1659,R2101012916,CV-6A,1405,A,T,0.356436,missense_variant,R2101012916,CV-6A
1660,R2101012916,CV-6A,1814,T,C,0.340659,synonymous_variant,R2101012916,CV-6A
1661,R2101012909,CV-6A,1387,C,A,0.160987,missense_variant,R2101012909,CV-6A


In [62]:
# Filter ParV-1A Host region
isnv_kept = isnv_kept[~((isnv_kept['CHROM']=='ParV-1A') & (isnv_kept['POS']>5000))]
isnv_kept.to_excel('iSNV_fullinfo_dropParVHost.xlsx', index=False)

In [59]:
result = []
for vsname, dfvs in isnv_kept.groupby('vsname'):
    
    fbvcfpath = '/jdfssz1/ST_HEALTH/P20Z10200N0206/renzirui/EA_redo/ViralStrain_PopGenetics/refiltering_test/filtered_annotated/{vsname}.snpeff.vcf'.format(vsname=vsname)
    fbvcfdf = vcf2dataframe(fbvcfpath)
    kept_fbsnlist = set(dfvs['SN']) & set(fbvcfdf.columns[9:])
    fbvcfvardict = { sn:list(fbvcfdf.loc[fbvcfdf[sn].apply(lambda x: x.startswith('1:')), 'POS']) for sn in kept_fbsnlist }
    
    pos_stat = dfvs.drop_duplicates(['sn','POS'])['POS'].value_counts()
    pos_stat_shareisnv = pos_stat[pos_stat > 1]
    pos_stat_singleisnv = pos_stat[pos_stat == 1]
    
    num_pos_stat_shareisnv = len(pos_stat_shareisnv)
    
    dfvs_singleisnv = dfvs[dfvs['POS'].isin(pos_stat_singleisnv.index)]
    for _, (pos, sn) in dfvs_singleisnv[['POS', 'SN']].iterrows():
        fbvcfvardict_pos = dict(filter(lambda x: pos in x[1], fbvcfvardict.items()))
        if len(set(fbvcfvardict_pos.keys()) | set([sn])) > 1:
            num_pos_stat_shareisnv += 1

    result.append([vsname, len(pos_stat), num_pos_stat_shareisnv])

resultdf = pd.DataFrame(result, columns=['vsname', 'num_pos', 'num_share_pos'])

In [60]:
resultdf

,vsname,num_pos,num_share_pos
0,AdV-2A,16,1
1,AstV-15A,12,0
2,AstV-9A,4,0
3,CV-12A,13,1
4,CV-14A,26,3
5,CV-1A,4,0
6,CV-2C,49,11
7,CV-3A,3,0
8,CV-4A,22,0
9,CV-6A,8,0


In [61]:
resultdf.num_share_pos.sum() / resultdf.num_pos.sum()

0.03994082840236687